In [131]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### **（10 pts）Split data into train and validation**

In [132]:
df = pd.read_csv('/content/gdrive/MyDrive/colab_input/data.csv')
from sklearn.model_selection import train_test_split
(train_x_split, valid_x_split,train_y_split, valid_y_split) = train_test_split(
    df['src'],
    df['tgt'],
    train_size=0.8,
    random_state=64096106
)
df.head(5)

,src,tgt
0,24-16-13=,-5
1,25-11-2=,12
2,26+19+20=,65
3,26-25+27=,28
4,26+17*12=,230


### **（10 pts）Generate data**

In [133]:
def digit_gen():
    first = np.random.randint(0, 30)
    second = np.random.randint(0, 30)
    third = np.random.randint(0, 30)
    op1 = np.random.choice(['+', '-', '*'])
    op2 = np.random.choice(['+', '-', '*'])

    result = eval(str(first) + op1 + str(second) + op2 + str(third))
    ques = f"{first}{op1}{second}{op2}{third}="
    answ = result
    return ques, answ

digit_gen()

('12+8+23=', 43)

In [134]:
mydf['answ'] = mydf['answ'].astype(str)
len(mydf)
mydf.head()
mydf.shape
mydf.head()

,ques,answ
0,8+22-2=,28
1,17*0*18=,0
2,24-8+17=,33
3,8-9-14=,-15
4,14*19+12=,278


### **Tokenize the text (10 pts)**

In [135]:
char_to_id = {}
id_to_char = {}

char_to_id['<pad>'] = 0
char_to_id['<eos>'] = 1
id_to_char[0] = '<pad>'
id_to_char[1] = '<eos>'

for char in set(df['src'].str.cat()):
    ch_id = len(char_to_id)
    char_to_id[char] = ch_id
    id_to_char[ch_id] = char

vocab_size = len(char_to_id)
print('字典大小: {}'.format(vocab_size))
char_to_id

字典大小: 16


{'<pad>': 0,
 '<eos>': 1,
 '*': 2,
 '4': 3,
 '5': 4,
 '9': 5,
 '0': 6,
 '3': 7,
 '+': 8,
 '7': 9,
 '2': 10,
 '1': 11,
 '-': 12,
 '6': 13,
 '=': 14,
 '8': 15}

In [136]:
# 把資料集的所有資料都變成id
df['tgt']=df['tgt'].astype(str)
df['src_id'] = df['src'].apply(lambda text: [char_to_id[char] for char in list(text)] )+df['tgt'].apply(lambda text: [char_to_id[char] for char in list(text)]+ [char_to_id['<eos>']] )
df['tgt_id'] = df['tgt'].apply(lambda text: [char_to_id[char] for char in list(text)] + [char_to_id['<eos>']])
df[['src','tgt','src_id','tgt_id']].head()

,src,tgt,src_id,tgt_id
0,24-16-13=,-5,"[10, 3, 12, 11, 13, 12, 11, 7, 14, 12, 4, 1]","[12, 4, 1]"
1,25-11-2=,12,"[10, 4, 12, 11, 11, 12, 10, 14, 11, 10, 1]","[11, 10, 1]"
2,26+19+20=,65,"[10, 13, 8, 11, 5, 8, 10, 6, 14, 13, 4, 1]","[13, 4, 1]"
3,26-25+27=,28,"[10, 13, 12, 10, 4, 8, 10, 9, 14, 10, 15, 1]","[10, 15, 1]"
4,26+17*12=,230,"[10, 13, 8, 11, 9, 2, 11, 10, 14, 10, 7, 6, 1]","[10, 7, 6, 1]"


In [137]:
batch_size = 100
epochs = 1000
embed_dim = 256
hidden_dim = 256
lr = 0.0001
grad_clip = 1

In [138]:
class Dataset(torch.utils.data.Dataset):
  # Python中的Dataset都需要做初始化
    def __init__(self, sequences):
        self.sequences = sequences # 把data讀入
  # 取出一筆資料
    def __getitem__(self, index):
        x = self.sequences.iloc[index][:-1] # 取得src_id中除了<eos>以外的設為x
        y = self.sequences.iloc[index][1:]  # 取得src_id中除了'='以外的設為x
        equal = y.index(char_to_id['='])
        for i in range(equal + 1):
          y[i] = 0 # padding掉'='左邊的token
        return x, y
  # 裡面有多少筆資料
    def __len__(self):
        return len(self.sequences)

# collate function將batch轉換為 torch.tensor
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch] # 將src轉換為torch.tensor
    batch_y = [torch.tensor(data[1]) for data in batch] # 將tgt轉換為torch.tensor
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    eq = torch.LongTensor([torch.where(data == char_to_id['='])[0] for data in batch_x])

    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True, # shape=(batch_size, seq_len)
                                                  padding_value=char_to_id['<pad>'])

    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True, # shape=(batch_size, seq_len)
                                                  padding_value=char_to_id['<pad>'])

    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens ,eq

In [139]:
dataset = Dataset(df['src_id'])
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          collate_fn=collate_fn)

In [140]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [141]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()

        # Embedding層
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        # RNN層
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        #self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
        #                                hidden_size=hidden_dim,
        #                                batch_first=True)
        # output層
        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))

    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)

    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)

        # 假設有個tensor : tensor([[1, 2, 3, 4],
        #                        [9, 0, 0, 0]])
        # 輸出就是：PackedSequence(data=tensor([1, 9, 2, 3, 4]),
        #                         batch_sizes=tensor([2, 1, 1, 1]),
        #                         sorted_indices=None, unsorted_indices=None)
        # torch.nn.utils.rnn.pack_padded_sequence 會把batch當中的句子從長到短排序，建立如上所示的資料結構
        # 就像上一個例子一樣，RNN會先吃第一個batch內的第一個batch_size，看到這個地方的batch_size爲2，所以此時RNN會吃兩個token，輸出一個2Xhidden_dim的向量組
        # 然後看第二個batch_size, 此時爲1，少了一個，說明其中一個序列到頭了，那就取上一個輸出向量的第一個，再生成一個1Xhidden_dim的向量
        # [
        # [1,2,3],                data = [1,4,6,2,5,3]   output  data = [1p,4p,6p,2p,5p,3p]             [1p,2p,3p]
        # [4,5,0], => pack_padded_sequence => batch_sizes = [3,2,1] => RNN => batch_sizes = [3,2,1] => pad_packed_sequence => [4p,5p,0]
        # [6,0,0]         _                                                     [6p,0,0]
        # ]
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)

        batch_x, _ = self.rnn_layer1(batch_x)
        #batch_x, _ = self.rnn_layer2(batch_x)
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)
        batch_x = self.linear(batch_x)
        return batch_x

    def generator(self, start_char, max_len=14):
        char_list = ''
        char_list += id_to_char[ch_id]
        char_list = [char_to_id[char] for char in start_char]
        next_char = None
        # print(start_char)
        # print(char_list)
        char_list_tensor = torch.LongTensor(char_list).unsqueeze(0).to(device)

        # 生成的長度沒達到max_len就一直生
        while len(char_list) < max_len:
            x = self.embedding(char_list_tensor) # 將字符序列轉換為嵌入向量
            _, (ht, _) = self.rnn_layer1(x) # RNN第一層對嵌入向量進行處理
            #_, (ht, _) = self.rnn_layer2(x) # RNN第二層對嵌入向量進行處理
            y = self.linear(ht) # linear層

            next_char = np.argmax(y.detach().cpu().numpy())

            # 如果看到新的token是<eos>就說明生成結束了，就停下
            if next_char == char_to_id['<eos>']:
                break

            char_list.append(next_char)
            char_list_tensor = torch.LongTensor(char_list).unsqueeze(0).to(device)

        return [id_to_char[ch_id] for ch_id in char_list]

In [142]:
#torch.manual_seed(2)
model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)
model = model.to(device)

In [143]:
for i in range(5):
  ques,answ = digit_gen()
  print('ques = ',ques)
  print('answ = ',answ)
  pr = ''
  for token in model.generator(ques):
    pr += token
  print(pr)
  print('-----------------------------------------------------------------------------------')
  print()

ques =  8-23+19=
answ =  4
8-23+19=++++++
-----------------------------------------------------------------------------------

ques =  21*16*3=
answ =  1008
21*16*3=810+++
-----------------------------------------------------------------------------------

ques =  29-26*7=
answ =  -153
29-26*7=610+++
-----------------------------------------------------------------------------------

ques =  1-7+11=
answ =  5
1-7+11=0++++++
-----------------------------------------------------------------------------------

ques =  19+11*29=
answ =  338
19+11*29=+++++
-----------------------------------------------------------------------------------



In [144]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'], reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [145]:
from tqdm import tqdm
model.train()
i = 0
for epoch in range(1, epochs+1):
    process_bar = tqdm(data_loader, desc=f"epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens,eq in process_bar:

        optimizer.zero_grad()
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        batch_pred_y = batch_pred_y.view(-1, vocab_size)
        batch_y = batch_y.view(-1).to(device)

        loss = criterion(batch_pred_y, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip)
        optimizer.step()

        i+=1
        if i%10 == 0:
            process_bar.set_postfix(loss=loss.item())

    validation_process_bar = tqdm(range(100))
    bingo = 0
    for i in validation_process_bar:
      ques,answ = digit_gen()
      pr = ''
      for token in model.generator(ques):
        pr += token
      if(pr == ques + str(answ)):
        # print('QA = ',ques, answ,', Prediction = ',pr)
        bingo += 1
    print("accuray=" + str(bingo / 100))

100%|██████████| 100/100 [00:00<00:00, 397.77it/s]


accuray=0.01


100%|██████████| 100/100 [00:00<00:00, 366.19it/s]


accuray=0.01


100%|██████████| 100/100 [00:00<00:00, 390.93it/s]


...

accuray=0.71


100%|██████████| 100/100 [00:00<00:00, 371.70it/s]


accuray=0.48


100%|██████████| 100/100 [00:00<00:00, 370.27it/s]

accuray=0.57
